In [1]:
import pandas as pd
import numpy as np
import re

df4 = pd.read_json('tt1424474-tt0006950.json')
df3 =pd.read_json('tt0010401-tt0144534.json')
# 合併兩個檔案
df = pd.concat([df3, df4], axis=0)
# 檢查有無重複值
df.index[df.index.duplicated()]

Index([], dtype='object')

In [2]:
# 預算 只挑$ 後有(estimated) 取最大值
df1 = df.applymap(lambda x: re.sub(r'[^\d]', '', x.replace('(estimated)', '')) if isinstance(x, str) and '(estimated)' in x and x.startswith('$') else np.nan)
merged_budget = pd.concat([df1[col] for col in df1.columns], axis=0)
merged_budget.dropna(inplace=True)
merged_budget = merged_budget.groupby(merged_budget.index).max()

In [3]:
# 收入 只挑$ 後無(estimated)取最大值
df2 = df.applymap(lambda x: re.sub(r'[^\d]', '', x) if isinstance(x, str) and x.startswith('$') and '(estimated)' not in x else np.nan)
merged_revenue = pd.concat([df2[col] for col in df2.columns], axis=0)
merged_revenue.dropna(inplace=True)
merged_revenue = merged_revenue.groupby(merged_revenue.index).max()

In [4]:
# 合併預算及收入
imdb_table = pd.concat([merged_budget, merged_revenue], axis=1)
# 更名
imdb_table = imdb_table.rename(columns={0: 'merged_budget', 1: 'merged_revenue'})
# 列出全部都不為NA的資料
imdb_table[imdb_table.notna().all(axis=1)]

,merged_budget,merged_revenue
tt0012190,800000,9183673
tt0014624,351000,11233
tt0018455,200000,121107
tt0018528,217000,4841
tt0018773,900000,27425
...,...,...
tt9686708,35000000,2180452
tt9779516,12000000,9868521
tt9857522,1000000,25618
tt9893250,14000000,1351662


In [5]:
#載入80萬筆的raw data
raw_df = pd.read_json('rawMovieWithTheNumAndKaggle.json')

In [6]:
len(imdb_table)

24283

In [7]:
len(raw_df)

860248

In [24]:
#將raw_df中imdb_id與imdb_table的index一樣的merged，表名子為merged_df 
merged_df = pd.merge(raw_df, imdb_table, how='inner', left_on='imdb_id', right_index=True)
#將budget為零或nan的以merged_budget欄位填入 
merged_df['budget'] = merged_df.apply(lambda x: x['merged_budget'] if pd.isna(x['budget']) or x['budget'] == 0 else x['budget'], axis=1)
#將revenue為零或nan的以merged_revenue欄位填入 
merged_df['revenue'] = merged_df.apply(lambda x: x['merged_revenue'] if pd.isna(x['revenue']) or x['revenue'] == 0 else x['revenue'], axis=1)

In [25]:
merged_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline,merged_budget,merged_revenue
41,1094962,tt3498690,Sorority of the Damned,,1200000,NaN,en,,0.000,0,...,[],False,A reluctant handy man is pitted against evil w...,0.600,[United States of America],0,Post Production,,1200000,NaN
43,1094960,tt9465024,Camp Terror,,20000,NaN,en,,0.000,0,...,[Showcase Productions],False,The Quiet town of Forrest Lakes was paralyzed ...,0.600,[United States of America],110,Released,,20000,NaN
218,1094770,tt15249688,The Squad: Rise of the Chicano Squad,,3300000,NaN,en,2022-06-30,0.000,0,...,[],False,In 1979 Houston TX. was a boom town fantasy ci...,0.600,[United States of America],0,Released,,3300000,NaN
278,1094701,tt27002895,A Corpse for Christmas,,6000,NaN,en,2023-07-17,0.000,0,...,[Blood Sick Productions],False,NecroPhiladelphia splatter punks celebrate Chr...,1.442,[United States of America],0,In Production,"A sexy, sadistic, metal fest shot on VHS!",6000,NaN
544,1094405,tt3302070,Mondo Sacramento 2,,10000,NaN,en,2013-10-10,0.000,0,...,[Desperate Visions Productions],False,Mondo Sacramento 2 features 6 true tales from ...,0.828,[United States of America],112,Released,Mondo Sacramento 2- All Crazy and all based on...,10000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860238,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,"[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play.",50000000,6496125
860241,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,"[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.,NaN,556354
860242,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...,NaN,49033349
860243,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.,2300000,269435


In [26]:
merged_df['revenue'][merged_df['revenue'].isna()]

41        NaN
43        NaN
218       NaN
278       NaN
544       NaN
         ... 
859383    NaN
859396    NaN
859397    NaN
859402    NaN
859460    NaN
Name: revenue, Length: 15271, dtype: object

In [27]:
merged_df['revenue'][merged_df['revenue'].isna()].index

Int64Index([    41,     43,    218,    278,    544,    590,    594,    599,
               857,   1073,
            ...
            859119, 859123, 859214, 859259, 859272, 859383, 859396, 859397,
            859402, 859460],
           dtype='int64', length=15271)

In [28]:
for index in merged_df['revenue'][merged_df['revenue'].isna()].index:
    merged_df.loc[index,'revenue'] = 0

for index in merged_df['budget'][merged_df['budget'].isna()].index:
    merged_df.loc[index,'budget'] = 0

In [29]:
merged_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline,merged_budget,merged_revenue
41,1094962,tt3498690,Sorority of the Damned,,1200000,0,en,,0.000,0,...,[],False,A reluctant handy man is pitted against evil w...,0.600,[United States of America],0,Post Production,,1200000,NaN
43,1094960,tt9465024,Camp Terror,,20000,0,en,,0.000,0,...,[Showcase Productions],False,The Quiet town of Forrest Lakes was paralyzed ...,0.600,[United States of America],110,Released,,20000,NaN
218,1094770,tt15249688,The Squad: Rise of the Chicano Squad,,3300000,0,en,2022-06-30,0.000,0,...,[],False,In 1979 Houston TX. was a boom town fantasy ci...,0.600,[United States of America],0,Released,,3300000,NaN
278,1094701,tt27002895,A Corpse for Christmas,,6000,0,en,2023-07-17,0.000,0,...,[Blood Sick Productions],False,NecroPhiladelphia splatter punks celebrate Chr...,1.442,[United States of America],0,In Production,"A sexy, sadistic, metal fest shot on VHS!",6000,NaN
544,1094405,tt3302070,Mondo Sacramento 2,,10000,0,en,2013-10-10,0.000,0,...,[Desperate Visions Productions],False,Mondo Sacramento 2 features 6 true tales from ...,0.828,[United States of America],112,Released,Mondo Sacramento 2- All Crazy and all based on...,10000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860238,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,"[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play.",50000000,6496125
860241,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,"[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.,NaN,556354
860242,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...,NaN,49033349
860243,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.,2300000,269435


In [30]:
filtered_df = merged_df.query('budget != 0 and revenue != 0').reset_index(drop=True)

In [31]:
#將merged_df中budget與revenue皆不為零或NaN的取出，
filtered_df = merged_df.query('budget != 0 and revenue != 0 and budget.notna() and revenue.notna()').reset_index(drop=True)

TypeError: unhashable type: 'numpy.ndarray'

In [32]:
len(filtered_df)

2652

In [33]:
#將不要的欄位去掉
filtered_df = filtered_df.drop(['merged_budget', 'merged_revenue'], axis=1)

In [34]:
filtered_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,keywords,genres,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline
0,1062413,tt16742890,Heretic,,2,1949,en,2021-12-09,2.000,1,...,"[evil spirit, occult, supernatural horror]","[Horror, Thriller]",[You Lucky Dog Production],False,A modern mystic using occult practices to comm...,2.100,[United States of America],87,Released,
1,40082,tt0104802,The Mambo Kings,,15500000,6742168,en,1992-02-28,6.263,93,...,[mambo],"[Drama, Music]","[Regency Enterprises, Alcor Films, Warner Bros...",False,The Mambo Kings tells the story of César and N...,10.075,"[France, United States of America]",104,Released,"With a suitcase full of song, Hearts filled wi..."
2,40048,tt0326965,In My Sleep,,1000000,9285,en,2010-04-23,5.500,31,...,"[surprise party, falling down stairs]","[Thriller, Mystery, Drama]",[],False,"A sleepwalking, sexaholic masseur wakes up, co...",3.834,[United States of America],90,Released,Sleepwalking can be deadly.
3,39934,tt0100049,Longtime Companion,,3000000,4609953,en,1989-10-11,7.114,56,...,"[aids, homophobia, politics, hiv, gay interest...","[Drama, Romance]","[American Playhouse, Companion Productions, Sa...",False,"During the summer of 1981, a group of friends ...",4.613,[United States of America],96,Released,…a motion picture for everyone
4,39914,tt0405977,Chaos,,1000000,20166,en,2005-08-10,4.600,25,...,"[rape, sadism, psychopath, rural area, torture...","[Horror, Thriller]",[Dominion Entertainment],False,Two girls heading to a rave take a detour to s...,5.578,[United States of America],74,Released,The most brutal movie ever made
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2647,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,[],"[Action, Thriller, Comedy]","[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play."
2648,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,[big smooth],"[Family, Adventure, Comedy]","[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.
2649,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[hippie, 1970s, based on true story, aftercred...",[Drama],"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...
2650,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,[postpartum depression],[Drama],"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.


In [35]:
#抓出raw_df裡的預算票房都有的7620筆資料存成clean_df
clean_df = raw_df.loc[(raw_df['budget'] != 0) & (raw_df['budget'].notnull()) & 
                      (raw_df['revenue'] != 0) & (raw_df['revenue'].notnull()) &
                      (raw_df['production_countries'].apply(lambda x: 'United States of America' in x))]

In [56]:
clean_df['imdb_id']

306       tt14022314
649       tt12057400
814       tt12580336
1755             NaN
4665      tt13309416
             ...    
860241     tt9765840
860242    tt10098448
860243    tt10925852
860244     tt1134854
860246     tt1480660
Name: imdb_id, Length: 7620, dtype: object

In [36]:
len(clean_df)

7620

In [53]:
test_df = pd.concat([clean_df, filtered_df], ignore_index=True)

In [55]:
test_df[(test_df.duplicated(subset=['revenue'])==True)]

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,keywords,genres,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline
1,1094287,tt12057400,Abscond,Abscond,500,500,en,2019-12-09,0.000,0,...,"[robbery, gun, bad guy]","[Drama, Mystery]",[],False,Two young thieves on the run reach a crossroad...,0.600,[United States of America],4,Released,
2,1094111,tt12580336,After Fire,After Fire,500,500,en,2020-06-21,0.000,0,...,"[fire, mystery, suspense]","[Drama, Mystery, Thriller]",[],False,"Grappling with the reality of their situation,...",0.600,[United States of America],6,Released,Truth Burns
13,1079470,NaN,The Ballad of the Punch-Down Kid,,2,10,en,2022-12-13,0.000,0,...,[],"[Drama, Comedy]",[],False,When an insecure student is confronted by a ps...,0.600,[United States of America],7,Released,
18,1078506,NaN,Night Walker : Bleeding For Love,,100,1000,en,,0.000,0,...,[],[Drama],"[CRASH-Force Studios, SPLATTERVISION]",False,A night in the life of a transvestite hooker a...,0.600,[United States of America],15,In Production,SEXY IS DANGEROUS
19,1073134,tt14294638,The First 280 Honest Words of My life,,250,1000,en,,0.000,0,...,[],[Drama],[ur girlfriends favorite films],False,A 14-year-old writer must write a short compos...,0.600,[United States of America],6,Released,Based on the mind of a crazy filmmaker.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,[],"[Action, Thriller, Comedy]","[Miramax, STX Entertainment, Toff Guy Films, T...",False,Special agent Orson Fortune and his team of op...,96.602,"[China, Turkey, United Kingdom, United States ...",114,Released,"In this operation, everyone has a part to play."
10268,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.850,10,...,[big smooth],"[Family, Adventure, Comedy]","[Almost Never Films, Fitz Family Films]",False,"Lena, a troubled youth takes a white lion cub ...",5.525,[United States of America],90,Released,Little friend. Big adventure.
10269,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.800,6,...,"[hippie, 1970s, based on true story, aftercred...",[Drama],"[Lionsgate, Kingdom Story Company]",False,"In the 1970s, aimless teenager Greg Laurie sea...",33.100,[United States of America],120,Released,When you open your heart...there's room for ev...
10270,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,[postpartum depression],[Drama],"[Sony Pictures, Maven Screen Media, Templehear...",False,"Julie is a new mom and children's book author,...",8.328,"[United Kingdom, United States of America]",105,Released,One breath at a time.


In [37]:
#將TMDB的7620與IMDB的2652筆合併
fin_imdb_merge_df = pd.concat([clean_df, filtered_df], ignore_index=True)

In [38]:
len(fin_imdb_merge_df)

10272

In [39]:
#刪除movie_id重複的資料
fin_imdb_merge_df.drop_duplicates(['movie_id'], inplace=True)

In [40]:
len(fin_imdb_merge_df)

9593

In [41]:
#將TMDB的7620與IMDB的2652筆合併
fin_imdb_merge_df2 = pd.concat([clean_df, filtered_df], ignore_index=True)

In [42]:
len(fin_imdb_merge_df2)

10272

In [43]:
#刪除imdb_id重複的資料
fin_imdb_merge_df2.drop_duplicates(['imdb_id'], inplace=True)

In [44]:
len(fin_imdb_merge_df2)

9425

In [45]:
fin_imdb_merge_df2.to_json('IMDB_merge_rawdata_0401.json')

In [52]:
fin_imdb_merge_df2['imdb_id'][fin_imdb_merge_df2['imdb_id']==0]

Series([], Name: imdb_id, dtype: object)

In [51]:
fin_imdb_merge_df2['imdb_id'][fin_imdb_merge_df2['imdb_id'].isna()]

Series([], Name: imdb_id, dtype: object)

In [50]:
fin_imdb_merge_df2.head(20)

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,keywords,genres,production_companies,adult,overview,popularity,production_countries,runtime,status,tagline
0,1094670,tt14022314,True Darkness: KEEP MOVING,True Darkness: KEEP MOVING,500,500,en,2021-10-31,0.000,0,...,"[ghost, haunt, scary]","[Horror, Mystery, Thriller]",[NOMAD COUTURE],False,"Jake can help people move on after death, but ...",0.600,[United States of America],7,Released,
1,1094287,tt12057400,Abscond,Abscond,500,500,en,2019-12-09,0.000,0,...,"[robbery, gun, bad guy]","[Drama, Mystery]",[],False,Two young thieves on the run reach a crossroad...,0.600,[United States of America],4,Released,
2,1094111,tt12580336,After Fire,After Fire,500,500,en,2020-06-21,0.000,0,...,"[fire, mystery, suspense]","[Drama, Mystery, Thriller]",[],False,"Grappling with the reality of their situation,...",0.600,[United States of America],6,Released,Truth Burns
3,1093009,NaN,NORMAN,,100,101,en,2023-03-23,0.000,0,...,"[high school, murder, coming of age, dating, t...",[Comedy],[],False,Norman Bumper isn’t your average party-goer. I...,1.959,[United States of America],8,In Production,BILLY BRYAN IS NORMAN BUMPER.
4,1089730,tt13309416,Dad Jokes,Dad Jokes,100,200,en,2021-05-11,9.000,1,...,"[marriage, joke telling, relationships, dad jo...","[Animation, Comedy]",[Jeaney Collects],False,A kid asks his father a question about relatio...,0.600,[United States of America],1,Released,"HI HORRIFIED, I'M DAD."
17,1078654,tt22487726,Kibble,Kibble,20,20,en,2022-10-31,0.000,0,...,"[horror, dog, short film, kibble]","[Mystery, Thriller, Horror]",[],False,A young man is dared by his friends to trek de...,0.600,[United States of America],8,Released,IT BETTER BE BITS
19,1073134,tt14294638,The First 280 Honest Words of My life,,250,1000,en,,0.000,0,...,[],[Drama],[ur girlfriends favorite films],False,A 14-year-old writer must write a short compos...,0.600,[United States of America],6,Released,Based on the mind of a crazy filmmaker.
20,1069847,,The Garage AxeMan,,100,1000,en,2021-07-23,10.000,1,...,[],"[Thriller, Horror]","[CRASH-Force Studios, SPLATTERVISION]",False,The 1ST short film from director Cooper R. Lan...,0.600,[United States of America],7,Released,
23,1066034,tt14242710,Lone Rider Animation,NaN,70,2000,pt,2011-01-10,10.000,1,...,[],"[Action, Animation, Adventure, Comedy, Crime, ...",[CULTZONE Filmes],False,,0.600,"[Brazil, United States of America, India, Nige...",0,Released,
25,1063680,tt14297960,Vergo: The Dennis Cut,Vergo: The Dennis Cut,1000,1000,en,2022-07-23,8.000,1,...,"[surrealism, found footage, psychological terr...","[Mystery, Horror, Drama]","[Cold Blood Studios, GraveStone Films, Jakeson...",False,Five friends go onto a streamyards live stream...,0.600,"[United Kingdom, United States of America]",114,Released,"Every second that goes by, These five men get ..."


In [57]:
perfect_df = pd.read_json('票房毛利預估_叫座/4_perfer_movie_7620.json'); perfect_df

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,Memorial Day,Juneteenth National Independence Day,Independence Day,Labor Day,Columbus Day,Veterans Day,Veterans Day (Observed),Thanksgiving,Christmas Day,country_is_USA_new
306,1094670,tt14022314,True Darkness: KEEP MOVING,True Darkness: KEEP MOVING,500,500,en,2021-10-31,,,...,False,False,False,False,False,False,False,False,False,True
649,1094287,tt12057400,Abscond,Abscond,500,500,en,2019-12-09,,,...,False,False,False,False,False,False,False,False,False,True
814,1094111,tt12580336,After Fire,After Fire,500,500,en,2020-06-21,,,...,False,False,False,False,False,False,False,False,False,True
1755,1093009,,NORMAN,,100,101,en,2023-03-23,,,...,False,False,False,False,False,False,False,False,False,True
4665,1089730,tt13309416,Dad Jokes,Dad Jokes,100,200,en,2021-05-11,9.0,1,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
860241,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.85,10,...,False,False,False,False,False,False,False,False,False,True
860242,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.8,6,...,False,False,False,False,False,False,False,False,False,True
860243,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,False,False,False,False,False,False,False,False,False,True
860244,29426,tt1134854,Survival of the Dead,,4000000,143191,en,2010-05-06,4.9,391,...,False,False,False,False,False,False,False,False,False,True


In [58]:
perfect_df['imdb_id'][perfect_df['imdb_id'].isna()]

Series([], Name: imdb_id, dtype: object)

In [60]:
new_fin_imdb = pd.concat([perfect_df, filtered_df], ignore_index=True); new_fin_imdb

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,Memorial Day,Juneteenth National Independence Day,Independence Day,Labor Day,Columbus Day,Veterans Day,Veterans Day (Observed),Thanksgiving,Christmas Day,country_is_USA_new
0,1094670,tt14022314,True Darkness: KEEP MOVING,True Darkness: KEEP MOVING,500,500,en,2021-10-31,,,...,False,False,False,False,False,False,False,False,False,True
1,1094287,tt12057400,Abscond,Abscond,500,500,en,2019-12-09,,,...,False,False,False,False,False,False,False,False,False,True
2,1094111,tt12580336,After Fire,After Fire,500,500,en,2020-06-21,,,...,False,False,False,False,False,False,False,False,False,True
3,1093009,,NORMAN,,100,101,en,2023-03-23,,,...,False,False,False,False,False,False,False,False,False,True
4,1089730,tt13309416,Dad Jokes,Dad Jokes,100,200,en,2021-05-11,9.0,1,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10268,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.85,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10269,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.8,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10270,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [63]:
new_fin_imdb[new_fin_imdb.duplicated(subset='movie_id')==False]

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,Memorial Day,Juneteenth National Independence Day,Independence Day,Labor Day,Columbus Day,Veterans Day,Veterans Day (Observed),Thanksgiving,Christmas Day,country_is_USA_new
0,1094670,tt14022314,True Darkness: KEEP MOVING,True Darkness: KEEP MOVING,500,500,en,2021-10-31,,,...,False,False,False,False,False,False,False,False,False,True
1,1094287,tt12057400,Abscond,Abscond,500,500,en,2019-12-09,,,...,False,False,False,False,False,False,False,False,False,True
2,1094111,tt12580336,After Fire,After Fire,500,500,en,2020-06-21,,,...,False,False,False,False,False,False,False,False,False,True
3,1093009,,NORMAN,,100,101,en,2023-03-23,,,...,False,False,False,False,False,False,False,False,False,True
4,1089730,tt13309416,Dad Jokes,Dad Jokes,100,200,en,2021-05-11,9.0,1,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9588,712128,tt11105790,Broken Keys,,2300000,9516,ar,2021-05-02,7.1,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9589,710356,tt5616176,2 Hearts,,15000000,522937,en,2020-10-16,7.471,434,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9590,708352,tt5768992,The Academy of Magic,,12000000,391806,en,2020-07-16,7.318,44,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9591,701121,tt12311866,Children of the Corn,,800000,457313,en,2020-10-23,7.0,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [65]:
new_fin_imdb[new_fin_imdb.duplicated(subset='movie_id', keep=False)==True]

,movie_id,imdb_id,title,en_title,budget,revenue,original_language,release_date,avg_rating,vote_count,...,Memorial Day,Juneteenth National Independence Day,Independence Day,Labor Day,Columbus Day,Veterans Day,Veterans Day (Observed),Thanksgiving,Christmas Day,country_is_USA_new
6878,1060455,tt8692378,After,,4000,12138565,en,2018-10-10,,,...,False,False,False,False,False,False,False,False,False,True
6879,39845,tt1125929,Love Ranch,,25000000,137885,en,2010-06-30,5.7,45,...,False,False,False,False,False,False,False,False,False,True
6880,39437,tt0120603,Beloved,,53000000,22852487,en,1998-10-16,5.653,85,...,False,False,False,False,False,False,False,False,False,True
6881,39401,tt1070858,The Secrets of Jonathan Sperry,,1000000,1355079,nl,2008-04-27,6.9,14,...,False,False,False,False,False,False,False,False,False,True
6882,39180,tt0250720,See Spot Run,,16000000,33357476,en,2001-03-02,5.89,191,...,False,False,False,False,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10267,739405,tt7985704,Operation Fortune: Ruse de Guerre,,50000000,32000000,en,2023-01-04,6.806,85,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10268,722799,tt9765840,Lena and Snowball,Lena and Snowball,400000,8756,en,2021-07-08,4.85,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10269,719256,tt10098448,Jesus Revolution,,15000000,33505612,en,2023-02-23,5.8,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10270,714195,tt10925852,A Mouthful of Air,,2300000,212833,en,2021-10-29,6.441,51,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
